In [1]:
# libraries
from datetime import datetime
import time
import pandas as pd
from pandas import DataFrame
import plotly.express as px
from Kafka import KafkaReader
from IPython.display import clear_output
from statics import emoji_dict, night_icons, wind_tips, roundtempto10, temp_dict, rain_dict, snow_dict, all_tips

### Aufgabe 6 Infographik automatisiert

In [4]:
def get_dataframe() -> DataFrame:
    # initializing the kafka reader
    reader = KafkaReader('weather.forecast')
    # getting the data from the cleaned topic
    data = reader.retrieve()
    # converting the received list in dataframe
    dataframe = pd.json_normalize(data)
    # 'treating' the weather data because its a list of nested dictionaries in the list of nested dictionaries 
    weather = dataframe['weather'].apply(lambda x: x[0])
    df2 = pd.json_normalize(weather)
    # concatenating the two data frames
    dataframe = pd.concat((dataframe, df2), axis=1)
    # dropping the column 'weather' from the original dataframe
    dataframe = dataframe.drop('weather', axis=1)

    # cleaning data: passed dates, that are still stored in the kafka topic, are to be filtered out
    now = datetime.timestamp(datetime.now())
    dataframe = dataframe.loc[dataframe['dt']>now]
    
    # doubled weather forecast data to be eliminated   
    dataframe['compare'] = dataframe['dt_txt'] + dataframe['city']
    dataframe = dataframe.drop_duplicates(subset=['compare'], keep='last')
    
    dataframe['displayed_text'] = dataframe['main.temp'].round(1).astype(str) + " °C"
    
    # adding icons / emojis depending on the weather condition to the dataframe
    dataframe['emojis'] = dataframe['icon'].map(emoji_dict)
    
    # adding tipps for the weather
    dataframe['rounded_temp'] = roundtempto10(dataframe['main.temp']).astype(int)
    dataframe['temp_tips'] = dataframe['rounded_temp'].map(temp_dict).fillna('')
    dataframe['snow_tips'] = dataframe['icon'].map(snow_dict).fillna('')
    dataframe['rain_tips'] = dataframe['id'].map(rain_dict).fillna('')
    dataframe['wind_tips'] = dataframe.apply(lambda x: wind_tips(x), axis=1).fillna('')
    dataframe['night_tips'] = ["It's bedtime" if x in night_icons else '' for x in dataframe['icon']]
    dataframe['Tips'] = dataframe.apply(lambda x: all_tips(x), axis=1)
    # dropping the unused columns from the dataframe
    dataframe = dataframe.drop(['temp_tips', 'snow_tips', 'rain_tips', 'wind_tips', 'night_tips', 'rounded_temp'], axis=1)
    
    return dataframe


def create_map_realtime() -> None:
    # getting the dataframe for the map
    forecasts = get_dataframe()

    # set all makers to the same size
    marker_max = 25
    
    hovertemplate = '<b>%{customdata[0]}</b><br>'+ '<br>%{customdata[1]}, %{customdata[2]}<br>'+ 'Feels like: %{customdata[5]}<br>' + 'Description: %{customdata[3]}' + '<br>Tip: %{customdata[4]}<br>'
    
    # making a map with plotly scatter mapbox 
    fig = px.scatter_geo(forecasts,
                         lat="coordinates.lat",
                         lon="coordinates.lon",
                         text="displayed_text",
                         hover_name="city",
                         # defining general details for the map
                         opacity=1,
                         scope='europe',
                         width=900,
                         height=600,
                         # setting the color + color range
                         color="main.temp",
                         color_continuous_scale="rdbu_r",
                         range_color=[-15,45],
                         # defining the attributes for the animation with the slider
                         animation_frame='dt_txt',
                         animation_group='city',
                         # defining the displayed data when hovering over the markers
                         hover_data = ['city', 'emojis', 'displayed_text', 'description', 'Tips', 'main.feels_like'],
                         # changing the name of the hover labels / text
                         labels={
                             "main.temp": "Temperature in °C",                       
                             "dt_txt": "Forecast for"
                         })
                        
    # setting the margin, the zoom & 'location' of the map and the size of the font in the map  
    fig.update_layout(
        margin={"r":0,"t":50,"l":0,"b":0},
        geo = dict(
            projection_scale=4,
            center=dict(lat=48.78, lon=9.17), 
        ),
        font=dict(
            size=8
        ),
        title = '<b>Weather Forecast Map</b><br>(Hover over the cities for more information)'
    )
    # defining a template for the layout / design of the map
    fig.update_geos(
        resolution=50,
        showcoastlines=True, coastlinecolor="black",
        showland=True, landcolor= "#CAEEC2",
        showocean=True, oceancolor="#d8e8f1"
    )
    # setting the size of the markers to marker_max
    fig.update_traces(marker={'size': marker_max}, hovertemplate = hovertemplate) 
    
    # passing the hovertemplate to every frame of the slider
    for f in fig.frames:
            f.data[0].hovertemplate = hovertemplate

    # showing the figure / map
    fig.show()  

In [3]:
# updating the figure time based and clearing the output every 15 secs
while True:
    clear_output(wait=True)
    create_map_realtime()
    time.sleep(15)

KeyboardInterrupt: 